In [4]:
import numpy as np
import pandas as pd
import math
# The operator module exports a set of efficient functions 
#corresponding to the intrinsic operators of Python
import operator

In [3]:
data = pd.read_csv("IRIS.csv")
data.head()
# TO do add header manually

,5.1,0.222222222,3.5,0.625,1.4,0.06779661,0.2,0.041666667,setosa
0,4.9,0.166667,3.0,0.416667,1.4,0.067797,0.2,0.041667,setosa
1,4.7,0.111111,3.2,0.500000,1.3,0.050847,0.2,0.041667,setosa
2,4.6,0.083333,3.1,0.458333,1.5,0.084746,0.2,0.041667,setosa
3,5.0,0.194444,3.6,0.666667,1.4,0.067797,0.2,0.041667,setosa
4,5.4,0.305556,3.9,0.791667,1.7,0.118644,0.4,0.125000,setosa


In [171]:
# create and shuffle datset
from sklearn.utils import shuffle
dataset = np.array(data)
dataset = shuffle(dataset)

dataset_X , dataset_y = dataset[: , 0:8] , dataset[: , -1]
dataset_X.shape , dataset_y.shape

x_train , y_train , x_val , y_val = dataset_X[0:90] , dataset_y[0:90] , dataset_X[90:] , dataset_y[90:]

In [73]:
def euclideanDistance(data1 , data2):
    features = len(data1)  # data1.shape : (4,)
    distance = 0
    
    for i in range(features):
        distance += np.square(data1[i] - data2[i])
    
    return np.sqrt(distance)

In [74]:
euclideanDistance(x_train[0] ,x_val[0])

5.5242591541184565

In [105]:
def kNN(trainingSet , testInstance , k , verbose=False):
    """
    kNN Model
    @params{trainingSet : 90% of IRIS.csv,
        testInstance : single data from remainign set,
        k : number of neighbours for prediction}
    """
    distance = {}
    sort = {}
    
    # Calculating euclidean distance between each row 
    # of training data and test data
    
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance , trainingSet[x])
    
        distance[x] = dist 
        
    # Sorting them on the basis of distance
#     if(verbose):
#         print("Distance :" , distance)
#         print()
        
        
    sorted_dist = sorted(distance.items() , key = operator.itemgetter(1))
    
    if(verbose):
        print("Sorted Distance from ALL traning Dataset :",sorted_dist)
        
    neighbours = []
    
    # Extracting top k neighbors
    for i in range(k):
        neighbours.append(sorted_dist[i][0])
    
    classVotes = {}
    
    
     # Calculating the most freq class in the neighbors
    
    for i in range(len(neighbours)):
        response = y_train[neighbours[i]]
        
        if response in classVotes:
            classVotes[response] +=1
            
        else:
            classVotes[response] = 1
            
            
    sortedVotes = sorted(classVotes.items() , key = operator.itemgetter(1) , reverse= True)
    
    if(verbose):
        print("All votes of neighbours : " , sortedVotes)
        
    
    return(sortedVotes[0][0] , neighbours)

In [99]:
result , neigh = kNN(x_train[0:10] , x_val[0] , 3)

In [196]:
def checkAcc(validate_y , k , verbose=False):
    """
    @params{y_val : 10% of IRIS.csv labels,
        k : number of neighbours for prediction}    
    """
    
    results = []
    neighbours = []
    for i in range(len(validate_y)):
        # Running KNN model
        result,neigh = kNN(x_train, x_val[i], k , verbose)
        results.append(result)
        neighbours.append(neigh)
        
#     if True:
#             print("Predicted Labels : " , results)
#             print("Actual Labels : " , validate_y)
    
    accuracy = np.sum(results == validate_y)/len(validate_y)
    
    if verbose :
        for i in range(len(validate_y)):
            if results[i]!=validate_y[i]:
                print("Predicted Labels : " , results[i])
                print("Actual Labels : " , validate_y[i])
                print("Neighbours used is : " , [y_train[name] for name in neighbours[i]])
        
    return accuracy , results

In [197]:
acc , _ = checkAcc(y_val , 5)

In [203]:
# Plotting Accuracy vs learning rate
import matplotlib.pyplot as plt
def plot_acc(k):
    accuracy = []
    for i in range(1,k):
        acc  , _ = checkAcc(y_val , i)
        accuracy.append(acc)

    k_val = [i for i in range(1 , k)]
#     print(accuracy)
    plt.plot(k_val, accuracy, 'ro')
    plt.axis([0, k, 0, 2])
    plt.show()

In [236]:
"""
Comparing our model with scikit-learn
"""

from sklearn.neighbors import KNeighborsClassifier
k=5
knn = KNeighborsClassifier(n_neighbors=k)

knn.fit(x_train , y_train)

pred = knn.predict(x_val)
neigh = knn.kneighbors(x_val)[1][: ,0:k]
print(neigh)

acc = np.sum(pred == y_val)/len(y_val)

print(acc)

[[31 17 89  5 12]
 [33 71  8 18 36]
 [87 78 27 40 10]
 [89 17 20 31 16]
 [63 70 55 65 51]
 [39 45 66 15 26]
 [76 55 65 11 70]
 [55 76 65 70  0]
 [63 51 70 32 65]]
1.0
